## 神经网络的数学基础

### 初识神经网络

神经网络相比与SVM、决策树等浅层特征表示的机器学习方法，具有更加强大的特征表达能力。

先从一个简单的例子：“手写数字识别” 开始。

问题描述：

将手写数字的灰度图像（$28 pixel \times 28 pixel$）划分到10个类别中（0～9）。这里将采用MNIST数据集，该数据集中包含60000张训练图像和10000张测试图像，由美国国家标准与技术研究院在20世纪80年代收集得到。

In [1]:
# 从Keras中加载MNIST数据集
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

/home/pzhang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_train.shape

(60000, 28, 28)

In [3]:
y_train # labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

接下来的工作流程如下：

1. 将训练数据（X_train，y_train）输入神经网络
2. 神经网络将图像和标签关联在一起
3. 将训练好的模型在测试集（X_test）上作预测，并与y_test比较来验证

In [4]:
from keras import models, layers

network = models.Sequential()
network.add(layers.Dense(512, activation="relu", input_shape=(28 * 28, )))
network.add(layers.Dense(10, activation="softmax", input_shape=(512, )))

神经网络的核心组件就是网络层（layer）了，其作用相当于数据过滤器。每层从输入中提取出更有助于我们解决问题的信息和数据。大多数的深度学习都是将简单的层链接起来，以实现渐进式的数据蒸馏（data distillation）。

本例子中的网络包含2个`Dense`层，它们是密集连接（也叫全连接）的神经层。第二层是一个10路`softmax`层，它将返回一个由10个概率值（和为1）组成的数组，表示当前数字的图像属于10个数字中某一个的概率。

想要训练网络，还需要对网络进行编译，确定三个参数：

1. 损失函数（loss function）
2. 优化器（optimizer）
3. 评定指标（metric）

In [5]:
network.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

在进行训练之前，还需要对数据进行预处理，将其变为网络所需的输入shape，并缩放到区间[0, 1]之间。

In [6]:
# 数据预处理
X_train = X_train.reshape(-1, 28*28)
X_train = X_train.astype('float32') / 255

X_test = X_test.reshape(-1, 28*28)
X_test = X_test.astype('float32') / 255

In [7]:
# one-hot 编码
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

现在调用网络的`fit()`方法来完成对训练集的学习

In [10]:
network.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 1s 14us/step - loss: 0.0293 - acc: 0.9908
Epoch 2/5
60000/60000 [==============================] - 1s 14us/step - loss: 0.0221 - acc: 0.9934
Epoch 3/5
60000/60000 [==============================] - 1s 14us/step - loss: 0.0172 - acc: 0.9951
Epoch 4/5
60000/60000 [==============================] - 1s 14us/step - loss: 0.0130 - acc: 0.9965
Epoch 5/5
60000/60000 [==============================] - 1s 14us/step - loss: 0.0100 - acc: 0.9970


训练过程中，显示的loss和acc分别为网络在训练集上的损失和精度大小。训练完成后，我们可以利用测试集对学到的模型进行测试

In [11]:
test_loss,  test_acc = network.evaluate(X_test, y_test)

print("test_acc: ", test_acc)

10000/10000 [==============================] - 0s 14us/step
test_acc:  0.9809


这样就完成了我们的第一个神经网络的学习和评估测试过程。